# Giesekus Creep: Strain Response Under Constant Stress

## Protocol Overview: Creep and Creep Compliance

In a creep test, we apply a constant stress σ₀ at t = 0 and measure the strain γ(t) over time. The **creep compliance** J(t) = γ(t)/σ₀ characterizes the material's time-dependent deformability.

**Key physics for Giesekus:**  
For a Maxwell fluid (linear viscoelastic):
$$
J(t) = \frac{1}{G}\left(1 + \frac{t}{\lambda}\right) = J_0 + \frac{t}{\eta_0}
$$

For the Giesekus model, the creep response is **nonlinear** due to the α·τ·τ term:
- **Low stress** (σ₀ ≪ G): Response approaches linear Maxwell
- **High stress** (σ₀ ~ G): Nonlinearity accelerates creep via anisotropic drag

The strain response is computed by solving the tensorial ODE with constant shear stress τ_xy = σ₀ (stress-controlled closure).

**Time scales:**
- **t ≪ λ**: Elastic response, γ ≈ σ₀/G (instantaneous compliance J₀ = 1/G)
- **t ≫ λ**: Viscous flow, γ ≈ σ₀·t/η₀ (steady-state slope dJ/dt = 1/η₀)

## Learning Objectives

1. Fit the Giesekus model to real mucus creep compliance data
2. Understand creep behavior in viscoelastic fluids (elastic → viscoelastic → viscous)
3. Extract retardation spectrum from creep response
4. Compare Giesekus creep to Maxwell (single exponential)
5. Use ODE-based simulation via diffrax for stress-controlled protocols

## Prerequisites

- Basic RheoJAX usage (basic/01-maxwell-fitting.ipynb)
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Estimated Runtime

- **Fast demo** (NUM_CHAINS=1, NUM_SAMPLES=500): ~3-5 minutes
- **Full run** (NUM_CHAINS=4, NUM_SAMPLES=2000): ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import gc
import json
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.giesekus import GiesekusSingleMode

# Add examples root to path for shared utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# Suppress equinox DeprecationWarning (jax.core.mapped_aval/unmapped_aval deprecated in JAX 0.8+)
warnings.filterwarnings("ignore", message=".*jax\\.core\\.(mapped|unmapped)_aval.*", category=DeprecationWarning)

# CI/demo toggle: True = fast (fewer samples), False = production (full inference)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: Creep in Viscoelastic Fluids

### The Experiment

In a creep test, we apply a constant stress σ₀ at t = 0 and measure the strain γ(t) over time.

### Creep Compliance

The creep compliance is defined as:
$$
J(t) = \frac{\gamma(t)}{\sigma_0}
$$

For a Maxwell fluid (linear viscoelastic):
$$
J(t) = \frac{1}{G} + \frac{t}{\eta} = \frac{1}{G}\left(1 + \frac{t}{\lambda}\right)
$$

where G = η/λ is the elastic modulus.

### Giesekus Creep

For the Giesekus model, the creep response is **nonlinear** due to the τ·τ term. At low stresses (σ₀ << G), the response approaches Maxwell. At high stresses, the nonlinearity accelerates creep.

The strain response is computed by solving the tensorial ODE:
$$
\boldsymbol{\tau} + \lambda \stackrel{\nabla}{\boldsymbol{\tau}} + \frac{\alpha \lambda}{\eta_p} \boldsymbol{\tau} \cdot \boldsymbol{\tau} = 2\eta_p \mathbf{D}
$$

subject to constant shear stress τ_xy = σ₀.

### Key Features

| Time Scale | Behavior |
|------------|----------|
| t << λ | Elastic response: γ ≈ σ₀/G |
| t >> λ | Viscous flow: γ ≈ σ₀·t/η |
| t ~ λ | Transition: viscoelastic creep |

## 3. Load Data

We use mucus creep compliance data. Mucus is a biopolymer gel that exhibits complex viscoelastic behavior.

In [ ]:
# Load mucus creep data
data_path = os.path.join("..", "data", "creep", "biological", "creep_mucus_data.csv")

# Tab-separated with header
raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
time_data = raw[:, 0]  # Time (s)
J_data = raw[:, 1]  # Creep compliance (1/Pa)

print(f"Loaded {len(time_data)} data points")
print(f"Time range: [{time_data.min():.2f}, {time_data.max():.1f}] s")
print(f"J range: [{J_data.min():.2e}, {J_data.max():.2e}] 1/Pa")

In [ ]:
# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(time_data, J_data, "ko", markersize=5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Creep compliance J(t) [1/Pa]")
ax1.set_title("Mucus Creep (Linear Scale)")
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(time_data, J_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Creep compliance J(t) [1/Pa]")
ax2.set_title("Mucus Creep (Log-Log Scale)")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

We fit the Giesekus model to the creep compliance data.

In [ ]:
# Create and fit Giesekus model
model = GiesekusSingleMode()

# For creep, we need to specify the applied stress
# Assume σ = 1 Pa for compliance normalization (J = γ/σ)
sigma_applied = 1.0  # Pa

# Estimate initial parameters from data to ensure stable fitting
# G ~ 1/J(0), η ~ 1/slope(J) at long times, λ ~ η/G
G_est = 1.0 / J_data[0]
idx_late = len(time_data) // 2
slope = np.polyfit(time_data[idx_late:], J_data[idx_late:], 1)[0]
eta_est = 1.0 / max(slope, 1e-10)
lambda_est = eta_est / max(G_est, 1e-10)

# Set initial parameters
# CRITICAL: eta_s must be non-zero for creep ODE stability
model.parameters.set_value('eta_p', max(eta_est, 1.0))
model.parameters.set_value('lambda_1', max(lambda_est, 0.1))
model.parameters.set_value('alpha', 0.1)
model.parameters.set_value('eta_s', max(0.1, 0.01 * eta_est))  # Small but non-zero

print("Initial parameter estimates from data:")
print(f"  G_est = {G_est:.4g} Pa, η_est = {eta_est:.4g} Pa·s, λ_est = {lambda_est:.4g} s")

t0 = time.time()
model.fit(time_data, J_data, test_mode="creep", sigma_applied=sigma_applied, method='scipy')
t_nlsq = time.time() - t0

print(f"\nNLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
param_names = ["eta_p", "lambda_1", "alpha", "eta_s"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

# Derived quantities
eta_p = model.parameters.get_value("eta_p")
eta_s = model.parameters.get_value("eta_s")
lambda_1 = model.parameters.get_value("lambda_1")
alpha = model.parameters.get_value("alpha")

eta_0 = eta_p + eta_s
G_0 = eta_p / lambda_1

print(f"\nDerived quantities:")
print(f"  η₀ = η_p + η_s = {eta_0:.2f} Pa·s")
print(f"  G₀ = η_p/λ = {G_0:.2f} Pa")
print(f"  J₀ = 1/G₀ = {1/G_0:.4e} 1/Pa")

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
plot_nlsq_fit(
    time_data, J_data, model, test_mode="creep",
    param_names=param_names,
    log_scale=False,
    xlabel="Time [s]", ylabel="Creep compliance J(t) [1/Pa]",
    title="Creep Fit (Linear Scale)",
    ax=ax1,
    sigma_applied=sigma_applied,
)

# Log-log scale
plot_nlsq_fit(
    time_data, J_data, model, test_mode="creep",
    param_names=param_names,
    log_scale=True,
    xlabel="Time [s]", ylabel="Creep compliance J(t) [1/Pa]",
    title="Creep Fit (Log-Log Scale)",
    show_annotation=False,
    ax=ax2,
    sigma_applied=sigma_applied,
)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in param_names
}
print("Warm-start values:")
for k, v in initial_values.items():
    print(f"  {k}: {v:.4g}")

# FAST_MODE: quick CI validation; production: full inference
if FAST_MODE:
    NUM_WARMUP = 50
    NUM_SAMPLES = 100
    NUM_CHAINS = 1
else:
    NUM_WARMUP = 1000
    NUM_SAMPLES = 2000
    NUM_CHAINS = 4

gc.collect()

t0 = time.time()
result = model.fit_bayesian(
    time_data,
    J_data,
    test_mode="creep",
    sigma_applied=sigma_applied,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
    target_accept_prob=0.95,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
diag = result.diagnostics

print("Convergence Diagnostics")
print("=" * 55)
print(f"{'Parameter':>12s}  {'R-hat':>8s}  {'ESS':>8s}  {'Status':>8s}")
print("-" * 55)

for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    status = "PASS" if (r_hat < 1.05 and ess > 100) else "CHECK"
    print(f"{p:>12s}  {r_hat:8.4f}  {ess:8.0f}  {status:>8s}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

In [ ]:
# Full ArviZ diagnostics: trace, pair, forest, energy, autocorr, rank
display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)

## 6. Posterior Predictive Check

In [ ]:
# Posterior predictive with 95% credible intervals
posterior = result.posterior_samples

fig, ax = plot_posterior_predictive(
    time_data, J_data, model, result,
    test_mode="creep", param_names=param_names,
    n_draws=min(100, len(posterior[param_names[0]])),
    log_scale=True,
    xlabel="Time [s]", ylabel="Creep compliance J(t) [1/Pa]",
    title="Posterior Predictive Check",
    sigma_applied=sigma_applied,
)
display(fig)
plt.close(fig)

## 7. Parameter Summary

In [ ]:
# NLSQ vs Bayesian comparison
print("Parameter Comparison: NLSQ vs Bayesian")
print("=" * 70)
print(f"{'Param':>12s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>26s}")
print("-" * 70)

# Reset NLSQ values first
for name, val in initial_values.items():
    model.parameters.set_value(name, val)

for name in param_names:
    nlsq_val = initial_values[name]
    bayes_samples = posterior[name]
    median = float(np.median(bayes_samples))
    lo = float(np.percentile(bayes_samples, 2.5))
    hi = float(np.percentile(bayes_samples, 97.5))
    print(f"{name:>12s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")

## 8. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "giesekus", "creep")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates
nlsq_params = initial_values.copy()
with open(os.path.join(output_dir, "nlsq_params_creep.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
with open(os.path.join(output_dir, "posterior_creep.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")

## Key Takeaways

1. **Creep compliance** J(t) = γ(t)/σ₀ characterizes the strain response under constant stress. It reveals the transition from elastic (t ≪ λ) to viscous (t ≫ λ) behavior.

2. **Short-time behavior** (t ≪ λ) is elastic: γ ≈ σ₀/G, giving instantaneous compliance J₀ = 1/G = λ/η_p.

3. **Long-time behavior** (t ≫ λ) is viscous: γ ≈ σ₀·t/η₀, giving steady-state slope dJ/dt = 1/η₀ where η₀ = η_p + η_s.

4. **Giesekus vs Maxwell**: The quadratic α·τ·τ term introduces stress-dependent nonlinearity. At high stresses, Giesekus shows faster creep than Maxwell due to anisotropic drag.

5. **Biological materials** like mucus show complex viscoelastic behavior with multiple relaxation times, often requiring multi-mode models.

### Experimental Notes

- **Creep tests are sensitive to**:
  - Material aging and thixotropy (rest time matters)
  - Wall slip (use roughened surfaces)
  - Sedimentation (for particle suspensions)
  - Temperature drift (use thermal equilibration)

- **Low stress creep** probes **linear regime** (Giesekus ≈ Maxwell, α has minimal effect)
- **High stress creep** probes **nonlinear regime** (α effect visible, accelerated creep)

### Creep Recovery

After unloading (stress removed at t = t₁):
- **Elastic strain recovered**: Δγ_rec ≈ σ₀/G (instantaneous elastic recoil)
- **Permanent (viscous) strain**: γ_perm = γ(t₁) - Δγ_rec (unrecoverable flow)

### Applications

| Material | Typical J₀ [1/Pa] | Application |
|----------|------------------|-------------|
| Polymer melts | 10⁻⁶ – 10⁻⁴ | Processing windows, sagging |
| Biological gels | 10⁻³ – 10⁻¹ | Drug delivery, tissue mechanics |
| Soft solids | 10⁻⁴ – 10⁻² | Shelf stability, aging |

### Next Steps

- **NB 06**: Stress relaxation (faster-than-Maxwell decay from α·τ·τ dissipation)
- **NB 07**: LAOS (nonlinear oscillatory response, intracycle nonlinearity)

## Further Reading

**RheoJAX Documentation:**
- [Giesekus Model Reference](../../docs/source/models/giesekus/giesekus.rst) — Creep equations (§ Protocol-Specific Equations)
- [Giesekus Index](../../docs/source/models/giesekus/index.rst) — All supported protocols

**Key References:**
1. Giesekus, H. (1982). "A simple constitutive equation for polymer fluids based on the concept of deformation-dependent tensorial mobility." *J. Non-Newtonian Fluid Mech.*, 11, 69-109.

2. Bird, R.B., Armstrong, R.C., & Hassager, O. (1987). *Dynamics of Polymeric Liquids, Vol. 1.* Wiley. Chapter 4.

3. Macosko, C.W. (1994). *Rheology: Principles, Measurements, and Applications.* Wiley-VCH. Chapter 3.